In [1]:
import sqlite3
import os
import pandas as pd
from datetime import datetime
from pandas.io import sql
import numpy as np

In [2]:
conn = sqlite3.connect('stock_analytics.db')

### 設計股票基本數據儲存表格
schema        | 中文     | Data Type
--------------|---------|----------
stock_id      | 股號     |
date          | 日期     |
volume        | 成交股數
turnover_value| 成交金額
open          | 開盤價
high          | 最高價
low           | 最低價
close         | 收盤價
spread        | 漲跌價差
change_ratio  | 漲跌比例
transaction_no| 成交筆數
PE_ratio      | 本益比

In [4]:
conn.execute(
    '''CREATE TABLE STOCK_PRICE
       (ID INTEGER PRIMARY KEY   AUTOINCREMENT,
        stock_id       TEXT    NOT NULL,
        date           TEXT,
        volume         INT,
        turnover_value INT,
        open           REAL,
        high           REAL,
        low            REAL,
        close          REAL,
        spread         REAL,
        change_ratio   REAL,
        transactions   INT,
        PE_ratio        REAL,
        Timestamp DATETIME DEFAULT CURRENT_TIMESTAMP);''')

In [5]:
FOLDER = 'data'
file_names = os.listdir(FOLDER)
problem_list = []
counter = 0
for file_name in file_names[counter:1000]:
    if not file_name.endswith('.csv'):
            continue
    print("Handling File: " + str(counter) + "-" + file_name)
    df = pd.read_csv(FOLDER+'/'+file_name, header = None)
    if len(df.columns) == 9: 
        df.columns=['date', 'volume', 'turnover_value', 'open', 'high', 'low', 'close', 'spread', 'transactions']
        df["PE_ratio"] = np.nan
    elif len(df.columns) == 10:
        df.columns=['date', 'volume', 'turnover_value', 'open', 'high', 'low', 'close', 'spread', 'transactions', 'PE_ratio']    
    else:
        print(file_name, " -> have a column number problem!")
        problem_list = problem_list.append(file_name)
    df['stock_id'] = [file_name.split('.')[0]] * len(df)
    date_parts = df['date'].str.split("/", expand=True)
    date_parts[0] = pd.to_numeric(date_parts[0])+1911
    df['date'] = date_parts[0].astype(str).str.cat(date_parts[1], sep='/').str.cat(date_parts[2], sep='/')
    df['date'] = pd.to_datetime(df['date'])
    df.replace(['--','---','X','----'], np.nan, inplace=True)
    df.replace(['除息','除權息', 'X0.00'], 0, inplace=True)
    df[['open', 'high', 'low', 'close', 'spread']] = df[['open', 'high', 'low', 'close', 'spread']].apply(pd.to_numeric)
    df['change_ratio'] = (df['close'] / (df['close'] - df['spread']) -1 )*100
    df = df.round({'change_ratio':2}) 
    df.set_index('date',inplace=True)
    sql.to_sql(df, name='STOCK_PRICE', con=conn, if_exists='append')
    conn.commit()
    counter += 1

Handling File: 0-030475.csv


In [119]:
df = pd.read_csv(FOLDER+'/2527.csv', header = None)

In [6]:
counter


1

In [91]:
df.dtypes


date              datetime64[ns]
volume                     int64
turnover_value             int64
open                     float64
high                     float64
low                      float64
close                    float64
spread                   float64
transactions               int64
PE_ratio                 float64
stock_id                  object
dtype: object

In [21]:
#file_name = file_names[0]
#
df.head()

,0,1,2,3,4,5,6,7,8,證券代號
0,99/06/04,0,0,--,--,--,--,NaN,0,030475
1,99/06/07,0,0,--,--,--,--,NaN,0,030475
2,99/06/08,30000,38400,1.28,1.28,1.28,1.28,-0.42,1,030475
3,99/06/09,30000,32400,1.08,1.08,1.08,1.08,-0.20,1,030475
4,99/06/10,6000,6540,1.09,1.09,1.09,1.09,0.01,2,030475


In [7]:
df.rename(columns=['date', 'volume', 'turnover_value', 'open', 'high', 'low', 'close', 'spread', 'transactions', 'stock_id'        
    
], inplace=True)

{'證券代號':'stock_id',
 '日期':'date',
 '成交股數':'volume',
 '成交筆數':'transactions',
 '成交金額':'turnover_value',
 '開盤價':'open',
 '最高價':'high',
 '最低價':'low',
 '收盤價':'close',
 '漲跌比例':'change_ratio',
 '漲跌價差':'spread',
 '本益比':'PE_ratio'}

In [16]:
df[[0]]

,0
0,99/06/04
1,99/06/07
2,99/06/08
3,99/06/09
4,99/06/10
5,99/06/11
6,99/06/14
7,99/06/15
8,99/06/17
9,99/06/18
